In [ ]:
#!/usr/bin/env python
# coding: utf-8

#
# Run this code as follows
#
# ex_multiclass.py <MODELNAME> <PDFNAME>
#
# where <MODELNAME> is the name of output model name (say my_model.h5)
# and <PDFNAME> is the name of the output PDF file with plots (say output.pdf)
#

#Import the necessary packages
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_curve,auc, confusion_matrix
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense
#from tensorflow.keras.utils import np_utils

import os
import sys
import warnings
warnings.filterwarnings('ignore')

#modelname = sys.argv[1]
#outputname = sys.argv[2]

modelname = "../ML_Output/3LTest_norm-5to5/multiclass_vll_3l.h5"
outputname = "../ML_Output/3LTest_norm-5to5/multiclass_vll_3l.pdf"

from matplotlib.backends.backend_pdf import PdfPages
pp = PdfPages(outputname)

#This is the full list of input variables in the text file
col_names=['LT_3L', 'HT_3L', 'ngenjet', 'j0pT', 'missinget', 'ST_3L', 'LTHT', 'LTMET', 'bestMOSSF_3L', 'trilepM', 'mt_0', 'mt_1', 'mt_2', 'dPhi_l0l1', 'dPhi_l1l2', 'dPhi_l0l2', 'maxdPhi', 'mindPhi', 'dR_l0l1', 'dR_l1l2', 'dR_l0l2', 'dRmin', 'dRmax', 'dPhi_l0MET', 'dPhi_l1MET', 'dPhi_l2MET', 'maxdPhi_LMET', 'mindPhi_LMET', 'dPhi_l0j0', 'dPhi_j0met', 'dR_l0j0', 'dR_j0met', 'pT0_3l', 'pT1_3l', 'pT2_3l', 'pT_tau_3l'];

#### For 2L1T: ['LT_2L1T', 'HT_2L1T', 'ngenjet', 'j0pT', 'missinget', 'ST_2L1T', 'LTHT', 'LTMET', 'dilepMass_2l1T', 'mT_2l1t_0', 'mT_2l1t_1', 'mtT_2l1t', 'dPhi_l0l1', 'dPhi_l0T', 'dPhi_l1T', 'maxdPhi', 'mindPhi', 'dR_l0l1', 'dR_l0T', 'dR_l1T', 'dRmin', 'dRmax', 'dPhi_l0MET', 'dPhi_l1MET', 'dPhi_TMET', 'maxdPhi_LMET', 'mindPhi_LMET', 'dPhi_l0j0', 'dPhi_j0met', 'dR_l0j0', 'dR_j0met', 'dPhi_Tj0', 'pT0_2l1t', 'pT1_2l1t', 'pT_tau_2l1t']

#cols = list(range(0,48))

#We start by using a small subset
#col_names=['Pt0','Pt1','Pt2','NBJet','MET','MaxDphi_LMet','MaxDphi_LL','MinDphi_LL','LLPairPt','Mt0','Mt1','Mt2']
#cols = [0,1,2,3,4,5,7,8,9,10,13,15,16,20,24,25] #26
#cols= list(range(len(col_names)))

cols = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


VLL_m500 = pd.read_csv('../VLLD_output/M500/VLLD_mu_M500_50k_SOM_3L_0.txt',sep=' ',index_col=None, usecols=cols,names=col_names)
VLL_m500_1 = pd.read_csv('../VLLD_output/M500/VLLD_mu_M500_50k_SOM_3L_1.txt',sep=' ',index_col=None, usecols=cols,names=col_names)
VLL_m500 = pd.concat([VLL_m500, VLL_m500_1])
VLL_m500['label']=0

VLL_m750 = pd.read_csv('../VLLD_output/M750/VLLD_mu_M750_50k_SOM_3L_0.txt',sep=' ',index_col=None, usecols=cols,names=col_names)
VLL_m750_1 = pd.read_csv('../VLLD_output/M750/VLLD_mu_M750_50k_SOM_3L_1.txt',sep=' ',index_col=None, usecols=cols,names=col_names)
VLL_m750 = pd.concat([VLL_m750, VLL_m750_1])
VLL_m750['label']=1

VLL_m1000 = pd.read_csv('../VLLD_output/M1000/VLLD_mu_M1000_50k_SOM_3L_0.txt',sep=' ',index_col=None, usecols=cols,names=col_names)
VLL_m1000_1 = pd.read_csv('../VLLD_output/M1000/VLLD_mu_M1000_50k_SOM_3L_1.txt',sep=' ',index_col=None, usecols=cols,names=col_names)
VLL_m1000 = pd.concat([VLL_m1000, VLL_m1000_1])
VLL_m1000['label']=2

#Reducing to 1000
#VLL_m500 = VLL_m500[:1000]
#VLL_m750 = VLL_m750[:1000]
#VLL_m1000 = VLL_m1000[:1000]

data = pd.concat([VLL_m500,VLL_m750,VLL_m1000])
X, y = data.values[:,:-1], data.values[:,-1]
#Here the y values, or labels are turned from 0,1,2 into
# one hot encoded values (1,0,0),(0,1,0),(0,0,1)
ohe_y = tf.keras.utils.to_categorical(y)

maxValues = X.max(axis=0)
minValues = X.min(axis=0)
print("Max values")
print(list(maxValues))
print("Min values")
print(list(minValues))
MaxMinusMin = X.max(axis=0) - X.min(axis=0)
normedX = 2*((X-X.min(axis=0))/(MaxMinusMin)) -1.0
X = normedX


X_train, X_test, y_train, y_test = train_test_split(X,ohe_y,test_size=0.5)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
n_features = X_train.shape[1]
print(f'The number of input variables is {n_features}')


model = Sequential()
model.add(Dense(128, activation='relu', kernel_initializer='he_normal', input_dim=n_features))
#model.add(Dense(128, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(64, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(32, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(16, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(3, activation='softmax'))

#compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train,y_train,epochs=20,batch_size=50,validation_data=(X_test,y_test),verbose=0)
                 
#Now we print the model summary to screen and save the model file
print('The NN architecture is')
model.summary()
model.save(modelname)


# Thats it. Now the rest of this file is just making various plots


# Let us start by making plots of the accuracy and loss as a function of epochs
# this tells us how the training went.
plt.figure(figsize=(7,5))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label = 'Val Accuracy')

#print("Training Accuracy:", history.history['accuracy'])
#print("Validation Accuracy:", history.history['val_accuracy'])

plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='upper left')
#plt.savefig('acc_v_epoch.png')
plt.savefig(pp, format='pdf')
plt.close()

plt.figure(figsize=(7,5))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label = 'Val Loss')

#print("loss:", history.history['loss'])
#print("val_loss:", history.history['val_loss'])

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim([0.001, 10])
plt.yscale('log')
plt.legend(loc='upper right')
#plt.savefig('loss_v_epoch.png')
plt.savefig(pp, format='pdf')
plt.close()

pp.close()
print(f'All done. Output is saved as {outputname}')


In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_curve,auc, confusion_matrix
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense
#from tensorflow.keras.utils import np_utils

import os
import sys
import warnings
warnings.filterwarnings('ignore')

In [60]:
col_names=['LT_3L', 'HT_3L', 'ngenjet', 'j0pT', 'missinget', 'ST_3L', 'LTHT', 'LTMET', 'bestMOSSF_3L', 'trilepM', 'mt_0', 'mt_1', 'mt_2', 'dPhi_l0l1', 'dPhi_l1l2', 'dPhi_l0l2', 'maxdPhi', 'mindPhi', 'dR_l0l1', 'dR_l1l2', 'dR_l0l2', 'dRmin', 'dRmax', 'dPhi_l0MET', 'dPhi_l1MET', 'dPhi_l2MET', 'maxdPhi_LMET', 'mindPhi_LMET', 'dPhi_l0j0', 'dPhi_j0met', 'dR_l0j0', 'dR_j0met', 'pT0_3l', 'pT1_3l', 'pT2_3l', 'pT_tau_3l'];
cols = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

VLL_m500 = pd.read_csv('../VLLD_output/M500/VLLD_mu_M500_50k_SOM_3L_0.txt',sep=' ',index_col=None, usecols=cols,names=col_names)
VLL_m750 = pd.read_csv('../VLLD_output/M750/VLLD_mu_M750_50k_SOM_3L_0.txt',sep=' ',index_col=None, usecols=cols,names=col_names)
VLL_m1000 = pd.read_csv('../VLLD_output/M1000/VLLD_mu_M1000_50k_SOM_3L_0.txt',sep=' ',index_col=None, usecols=cols,names=col_names)


VLL_m500['label']=0
VLL_m750['label']=1
VLL_m1000['label']=2

data = pd.concat([VLL_m500,VLL_m750,VLL_m1000], ignore_index = True)
X, y = data.values[:,:-1], data.values[:,-1]


#Here the y values, or labels are turned from 0,1,2 into
# one hot encoded values (1,0,0),(0,1,0),(0,0,1)
ohe_y = tf.keras.utils.to_categorical(y)

maxValues = X.max(axis=0)
minValues = X.min(axis=0)
print("Max values")
print(list(maxValues))
print("Min values")
print(list(minValues))
MaxMinusMin = X.max(axis=0) - X.min(axis=0)
normedX = 2*((X-X.min(axis=0))/(MaxMinusMin)) -1.0


Max values
[3853.49, 4858.7, 13.0, 2239.86, 1666.94, 8285.07, 7912.47, 4446.6, 3781.0, 4420.83]
Min values
[77.0744, 72.7997, 1.0, 41.6797, 1.00854, 200.637, 149.874, 127.837, 0.0, 85.6331]


In [61]:
meanVals = X.mean(axis=0)
stdVals = X.std(axis=0)

In [62]:
meanVals

array([ 875.83465957, 1209.23579305,    4.70868266,  498.28914897,
        270.80954887, 2355.87998227, 2085.0704349 , 1146.64418782,
        458.30561086,  937.35983963])

In [63]:
stdVals

array([396.62411436, 538.32312898,   1.28805558, 231.4083639 ,
       207.22071781, 927.24274784, 896.76963682, 457.2087577 ,
       386.5324777 , 470.12181848])

In [64]:
X.shape, y.shape

((10377, 10), (10377,))

In [65]:
abs((X-meanVals))/stdVals < 5 

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [66]:
(abs((X-meanVals))/stdVals < 5 ).sum()

103646

In [67]:
boolarr = (abs((X-meanVals))/stdVals < 5 )

In [68]:
boolarr.index(False)

AttributeError: 'numpy.ndarray' object has no attribute 'index'

In [69]:
boolarr.argmin(axis=0)

array([5120, 4239, 2651, 3890, 4564, 4239, 4239, 4564, 2119, 4011])

In [70]:
boolarr[5120]

array([False,  True,  True, False,  True, False, False, False, False,
       False])

In [71]:
boolarr[4239], boolarr[2651], boolarr[3890], boolarr[4564], boolarr[4239]

(array([ True, False,  True,  True,  True, False, False,  True,  True,
         True]),
 array([ True,  True, False,  True,  True,  True,  True,  True,  True,
         True]),
 array([ True,  True,  True, False,  True,  True,  True,  True,  True,
         True]),
 array([ True,  True,  True,  True, False,  True,  True, False,  True,
         True]),
 array([ True, False,  True,  True,  True, False, False,  True,  True,
         True]))

In [72]:
boolarr[4564], boolarr[2119], boolarr[4011]

(array([ True,  True,  True,  True, False,  True,  True, False,  True,
         True]),
 array([ True,  True,  True,  True,  True,  True,  True,  True, False,
         True]),
 array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        False]))

In [73]:
X[2651]

array([1149.51 , 3116.08 ,   12.   ,  750.568,  108.586, 4374.18 ,
       4265.6  , 1258.1  ,  319.705,  645.37 ])

In [74]:
X[4011]

array([2163.51 , 2533.67 ,    6.   , 1172.01 ,  396.224, 5093.41 ,
       4697.18 , 2559.74 ,  469.454, 3346.79 ])

In [75]:
mybool = np.array([[True, True, True], [False, True, True], [True, False, False], [True, True, True]])

In [76]:
mybool.argmin(axis=1)

array([0, 0, 1, 0])

In [77]:
mybool.argmax(axis=1)

array([0, 1, 0, 0])

In [78]:
mybool[mybool == True]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True])

In [79]:
mybool[mybool.sum(axis = 1) == mybool.shape[1]]

array([[ True,  True,  True],
       [ True,  True,  True]])

In [80]:
data[boolarr.sum(axis = 1) == boolarr.shape[1]]

,LT_3L,HT_3L,ngenjet,j0pT,missinget,ST_3L,LTHT,LTMET,bestMOSSF_3L,trilepM,label
0,617.693,1036.240,7,308.630,41.1043,1695.040,1653.930,658.798,530.804,715.380,0
1,355.932,338.643,2,295.199,282.5120,977.088,694.575,638.444,282.617,466.747,0
2,553.522,1066.850,6,477.566,358.0190,1978.390,1620.370,911.541,164.729,324.409,0
3,644.087,648.176,3,421.943,451.0930,1743.360,1292.260,1095.180,243.521,624.387,0
4,988.347,1053.630,4,501.318,214.2090,2256.190,2041.980,1202.560,290.667,983.948,0
...,...,...,...,...,...,...,...,...,...,...,...
10372,1185.500,1186.590,3,795.121,630.6950,3002.790,2372.100,1816.200,646.270,1645.640,2
10373,1040.240,1045.980,3,514.573,42.7216,2128.930,2086.210,1082.960,130.680,1057.040,2
10374,1280.210,2597.790,8,715.531,188.0380,4066.030,3877.990,1468.240,449.747,1043.840,2
10375,1071.680,1074.070,3,714.316,517.2770,2663.020,2145.750,1588.960,364.871,1068.630,2


In [81]:
ohe_y

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [82]:
data

,LT_3L,HT_3L,ngenjet,j0pT,missinget,ST_3L,LTHT,LTMET,bestMOSSF_3L,trilepM,label
0,617.693,1036.240,7,308.630,41.1043,1695.040,1653.930,658.798,530.804,715.380,0
1,355.932,338.643,2,295.199,282.5120,977.088,694.575,638.444,282.617,466.747,0
2,553.522,1066.850,6,477.566,358.0190,1978.390,1620.370,911.541,164.729,324.409,0
3,644.087,648.176,3,421.943,451.0930,1743.360,1292.260,1095.180,243.521,624.387,0
4,988.347,1053.630,4,501.318,214.2090,2256.190,2041.980,1202.560,290.667,983.948,0
...,...,...,...,...,...,...,...,...,...,...,...
10372,1185.500,1186.590,3,795.121,630.6950,3002.790,2372.100,1816.200,646.270,1645.640,2
10373,1040.240,1045.980,3,514.573,42.7216,2128.930,2086.210,1082.960,130.680,1057.040,2
10374,1280.210,2597.790,8,715.531,188.0380,4066.030,3877.990,1468.240,449.747,1043.840,2
10375,1071.680,1074.070,3,714.316,517.2770,2663.020,2145.750,1588.960,364.871,1068.630,2


In [84]:
data.loc[5], ohe_y[5]

(LT_3L            758.1890
 HT_3L            895.3660
 ngenjet            4.0000
 j0pT             467.0050
 missinget        351.4410
 ST_3L           2005.0000
 LTHT            1653.5600
 LTMET           1109.6300
 bestMOSSF_3L      90.8141
 trilepM          565.2790
 label              0.0000
 Name: 5, dtype: float64,
 array([1., 0., 0.]))

In [85]:
((X-meanVals))/stdVals

array([[-0.65084711, -0.32136051,  1.77889632, ..., -1.06700972,
         0.18756093, -0.47217515],
       [-1.31081959, -1.61723089, -2.10292374, ..., -1.11152768,
        -0.45452484, -1.00104446],
       [-0.8126401 , -0.26449875,  1.00253231, ..., -0.5142141 ,
        -0.75951344, -1.30381279],
       ...,
       [ 1.019543  ,  2.57940655,  2.55526033, ...,  0.70338944,
        -0.02214202,  0.22649483],
       [ 0.49378072, -0.25108673, -1.32655973, ...,  0.96742638,
        -0.24172512,  0.27922584],
       [ 0.60991082,  2.04316729,  0.2261683 , ...,  0.70677958,
         0.89343951,  0.8734548 ]])

In [89]:
#!/usr/bin/env python
# coding: utf-8

#
# Run this code as follows
#
# ex_multiclass_test.py <MODELNAME> <PDFNAME>
#
# where <MODELNAME> is the name of input model name (say my_model.h5)
# and <PDFNAME> is the name of the output PDF file with plots (say output.pdf)
#

#Import the necessary packages
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve,auc, confusion_matrix

import os
import sys
import warnings
warnings.filterwarnings('ignore')

#modelname = sys.argv[1]
#outputname = sys.argv[2]

modelname = "../ML_Output/3LTest_norm-5to5/multiclass_vll_3l.h5"
outputname = "../ML_Output/3LTest_norm-5to5/multiclass_vll_3l_testing.pdf"

from matplotlib.backends.backend_pdf import PdfPages
pp = PdfPages(outputname)

#First we put in the same columns here that we have in the mynn.py

col_names=['LT_3L', 'HT_3L', 'ngenjet', 'j0pT', 'missinget', 'ST_3L', 'LTHT', 'LTMET', 'bestMOSSF_3L', 'trilepM', 'mt_0', 'mt_1', 'mt_2', 'dPhi_l0l1', 'dPhi_l1l2', 'dPhi_l0l2', 'maxdPhi', 'mindPhi', 'dR_l0l1', 'dR_l1l2', 'dR_l0l2', 'dRmin', 'dRmax', 'dPhi_l0MET', 'dPhi_l1MET', 'dPhi_l2MET', 'maxdPhi_LMET', 'mindPhi_LMET', 'dPhi_l0j0', 'dPhi_j0met', 'dR_l0j0', 'dR_j0met', 'pT0_3l', 'pT1_3l', 'pT2_3l', 'pT_tau_3l'];

cols = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]





VLL_m500 = pd.read_csv('../VLLD_output/M500/VLLD_mu_M500_50k_SOM_3L_2.txt',sep=' ',index_col=None, usecols=cols,names=col_names)
VLL_m750 = pd.read_csv('../VLLD_output/M750/VLLD_mu_M750_50k_SOM_3L_2.txt',sep=' ',index_col=None, usecols=cols,names=col_names)
VLL_m1000 = pd.read_csv('../VLLD_output/M1000/VLLD_mu_M1000_50k_SOM_3L_2.txt',sep=' ',index_col=None, usecols=cols,names=col_names)

#Reducing to 1000
#VLL_m500 = VLL_m500[:1000]
#VLL_m750 = VLL_m750[:1000]
#VLL_m1000 = VLL_m1000[:1000]


alldfs = [VLL_m500,VLL_m750,VLL_m1000]
alldfnames = ['M500','M750','M1000']

#=====================================
#Now we must declare arrays to keep the max and min values that we got from ex_multiclass.py
#maxVals = np.array([[2.03079e+03, 1.13610e+03, 4.53007e+02, 4.00000e+00, 1.27904e+03, 3.14159e+00,
#                      3.14158e+00, 2.09007e+00, 2.28011e+03, 2.30414e+03, 1.23756e+03, 7.52082e+02]])
#minVals = np.array([[10.3434,    10.0244,    10.0001,     0.,         0.0885814,  0.0112305,
#                     0.,         0.,         0.0437537,  0.,         0.,         0.       ]])

'''
maxVals = np.array([[1360.896973, 1184.820679,  458.599152,    5.,       1613.865112,    3.141593,
                     3.141584,    2.090327, 1366.192383, 2078.448242, 2033.94812,  1022.813843]])
minVals = np.array([[2.9000189e+01,  1.0023091e+01,  1.0000089e+01,  0.0000000e+00,
                     5.7219000e-02,  4.4246000e-02,  1.6981000e-02,  0.0000000e+00,
                     -1.0000000e+00,  7.3600000e-04,  1.9700000e-04,  1.8100000e-04]])
'''

maxVals = np.array([[3853.49, 4858.7, 13.0, 2239.86, 2600.15, 8457.11, 8035.31, 4756.4, 3815.36, 4455.1]])
minVals = np.array([[77.0744, 37.4362, 1.0, 37.4362, 1.00582, 188.919, 125.084, 127.837, 0.0, 74.8925]])

meanVals = np.array([[875.9111615503876, 1209.6366846511628, 4.716327519379845, 499.2981335319767, 270.24741829118216, 2355.7952771802325, 2085.5478483042634, 1146.1585742248062, 461.2242037306202, 937.8816425968993]])
stdVals = np.array([[396.10768659831433, 536.2110115807294, 1.2795366474884335, 230.38747025602152, 207.5140772531654, 925.4488594207648, 894.8704288463225, 457.67349631934815, 385.30691898646177, 469.6052755141552]])

mymodel = tf.keras.models.load_model(modelname)
mymodel.load_weights(modelname)



mybins = np.arange(0,1.05,0.05)

#This function takes a dataframe and plots all three scores for it
def processdf(df,dfname):
    #normeddf = 2*( (df-minVals)/(maxVals-minVals) ) -1.0
    #df = normeddf

    ### New Normalisation:
    normedX = (df-meanVals)/stdVals # Normalising the values
    #boolarr = abs((df-meanVals))/stdVals < 5   # Creating Boolean array of whether the values are within the given range 
    #normedX = df[boolarr.sum(axis = 1) == boolarr.shape[1]] # Selecting only events which have all the values within 5 sigma
    df = normedX


    nnscore = mymodel.predict(df)
    plt.figure(figsize=(8,6))
    plt.hist(nnscore[:,0],bins=mybins,histtype='step',label='M500neuron',linewidth=3,color='xkcd:sky blue',density=False,log=False)
    plt.hist(nnscore[:,1],bins=mybins,histtype='step',label='M750neuron',linewidth=3,color='xkcd:red',density=False,log=False)
    plt.hist(nnscore[:,2],bins=mybins,histtype='step',label='M1000neuron',linewidth=3,color='xkcd:green',density=False,log=False)
    plt.legend(loc='upper center')
    plt.xlabel('Score',fontsize=20)
    plt.ylabel('Events',fontsize=20)
    plt.title(f'NN Output for {dfname}',fontsize=20)
    plt.xticks([0.0,0.2,0.4,0.6,0.8,1.0],fontsize=12)
    plt.yticks(fontsize=12)
    #plt.ylim([1, 1.1e5])
    #Save to file instead of individual image
    #plt.savefig('score'+dfname+'.png')
    plt.savefig(pp, format='pdf')
    plt.close()

#This function takes a list of dataframes and plots a specific score for it
mycols=['xkcd:sky blue','xkcd:red','xkcd:green']
labelnames = ['M500neuron','M750neuron','M1000neuron']
def processdflist(dflist,dfnames,proclabel):

    plt.figure(figsize=(8,6))
    for index, df in enumerate(dflist):
        #normeddf = 2*( (df-minVals)/(maxVals-minVals) ) -1.0
        #df = normeddf

        ### New Normalisation:
        normedX = (df-meanVals)/stdVals # Normalising the values
        #boolarr = abs((df-meanVals))/stdVals < 5   # Creating Boolean array of whether the values are within the given range 
        #normedX = df[boolarr.sum(axis = 1) == boolarr.shape[1]] # Selecting only events which have all the values within 5 sigma
        df = normedX
     
        nnscore = mymodel.predict(df)
        plt.hist(nnscore[:,proclabel],bins=mybins,histtype='step',label=dfnames[index],
                 linewidth=3,color=mycols[index],density=False,log=False)
    plt.legend(loc='upper center')
    plt.xlabel('Score',fontsize=20)
    plt.ylabel('Events',fontsize=20)
    plt.title(f'{labelnames[proclabel]} NN Output',fontsize=20)
    plt.xticks([0.0,0.2,0.4,0.6,0.8,1.0],fontsize=12)
    plt.yticks(fontsize=12)
    #plt.ylim([1, 1.1e5])
    #plt.savefig('score'+labelnames[proclabel]+'.png')
    plt.savefig(pp, format='pdf')    
    plt.close()
    
    
processdf(VLL_m500,'procM500')
processdf(VLL_m750,'procM750')
processdf(VLL_m1000,'procM1000')
processdflist(alldfs,alldfnames,0)
processdflist(alldfs,alldfnames,1)
processdflist(alldfs,alldfnames,2)


pp.close()
print(f'All done. Output is saved as {outputname}')


107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 398us/step
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 258us/step
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 246us/step
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 240us/step
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 229us/step
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 231us/step
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 231us/step
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 460us/step
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 247us/step
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 232us/step
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 229us/step
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 228us/step
All done. Output is saved as ../ML_Output/3LTest_norm-5to5/multiclass_vll_3l_testing.pdf
